# Advanced Flight Data Analysis Questions
This notebook contains mid to advanced level questions for flight data analysis using PySpark. Please provide your solutions in the code cells following each question.

In [ ]:
# import and spark session creation cell

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("DataFrame Example").getOrCreate()

df = spark.read.csv("")

## Question 1: Top 5 busiest airports by total passenger traffic
Find the top 5 airports (considering both origin and destination) with the highest total number of passengers.

In [ ]:
# Your solution here


## Question 2: Monthly trend of total flights
Calculate the total number of flights for each month across all years in the dataset. Show the results ordered by month.

In [ ]:
# Your solution here

## Question 3: Flight routes with consistently high seat occupancy
Identify flight routes where the average seat occupancy (Passengers / Seats) is greater than 80% across all flights.

In [ ]:
# Your solution here

## Question 4: Correlation between flight distance and seat occupancy
Analyze if there is any correlation between the flight distance and the average seat occupancy on the route.

In [ ]:
# Your solution here

## Question 5: Identify routes with significant seasonal variation in passenger numbers
Find routes where the number of passengers varies significantly between different quarters of the year.

In [ ]:
# Your solution here